In [2]:
import findspark
import pyspark
import pyspark.sql
import pyspark.sql.functions as F
import pyspark.ml as ml
from catboost import CatBoostClassifier

from itertools import chain

findspark.init('/opt/spark')

In [3]:
spark = (
    pyspark
    .sql
    .SparkSession
    .builder
    .config('spark.jars', '/usr/share/java/mysql-connector-java.jar')
    .master('local[8]')
    .appName('sber_inference')
    .getOrCreate()
)

spark

22/09/02 04:10:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/02 04:10:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/02 04:10:21 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [8]:
df = (
    spark
    .read
    .format('jdbc')
    .option('url', 'jdbc:mysql://localhost:3306/sber')
    .option('driver', 'com.mysql.cj.jdbc.Driver')
    .option('dbtable', 'neo')
    .option('user', 'sber')
    .option('password', 'sber65537')
    .load()
)

df = df.drop('name', 'orbiting_body', 'sentry_object')

feature_columns = df.columns
feature_columns.remove('hazardous')
feature_columns.remove('id')

model = CatBoostClassifier().load_model('catboost_clf')

@F.pandas_udf(returnType='id bigint, neo_prob float', 
              functionType=F.PandasUDFType.GROUPED_MAP)
def predict(df):
  df['neo_prob'] = model.predict_proba(df.loc[:, feature_columns])[:, 1]  
  return df.loc[:, ['id', 'neo_prob']]

prediction = (
  df
  .groupBy(F.spark_partition_id())
  .apply(predict)
)

/home/neoksei/.miniconda/envs/ml_mipt/lib/python3.9/site-packages/pyspark/sql/pandas/group_ops.py:98: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [10]:
(
    prediction
    .write
    .mode('overwrite')
    .format('jdbc')
    .option('url', 'jdbc:mysql://localhost:3306/sber')
    .option('driver', 'com.mysql.cj.jdbc.Driver')
    .option('dbtable', 'neo_probs')
    .option('user', 'sber')
    .option('password', 'sber65537')
    .save()
)